In [1]:
my_data =[['slashdot','USA','yes',18,'None'],
          ['google','France','yes',23,'Premium'],
          ['digg','USA','yes',24,'Basic'],
          ['kiwitobes','France','yes',23,'Basic'],
          ['google','UK','no',21,'Premium'],
          ['(direct)','New Zealand','no',12,'None'],
          ['(direct)','UK','no',21,'Basic'],
          ['google','USA','no',24,'Premium'],
          ['slashdot','France','yes',19,'None'],
          ['digg','USA','no',18,'None'],
          ['google','UK','no',18,'None'],
          ['kiwitobes','UK','no',19,'None'],
          ['digg','New Zealand','yes',12,'Basic'],
          ['google','UK','yes',18,'Basic'],
          ['kiwitobes','France','yes',19,'Basic']]

In [2]:
class decisionnode:
    def __init__(self , col = -1 , value = None , results = None , tb=None , fb = None):
        self.col = col
        self.value = value
        self.results = results
        self.tb = tb
        self.fb = fb

In [3]:
def divideset(rows , column , value):
    split_function = None
    
    if isinstance(value , int) or isinstance(value , float):
        #数值型数据
        split_function = lambda row: row[column] >= value
    else:
        #标称型数据
        split_function = lambda row: row[column] == value
        
    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]
    
    return (set1 , set2)

In [4]:
divideset(my_data , 2 , 'yes')

([['slashdot', 'USA', 'yes', 18, 'None'],
  ['google', 'France', 'yes', 23, 'Premium'],
  ['digg', 'USA', 'yes', 24, 'Basic'],
  ['kiwitobes', 'France', 'yes', 23, 'Basic'],
  ['slashdot', 'France', 'yes', 19, 'None'],
  ['digg', 'New Zealand', 'yes', 12, 'Basic'],
  ['google', 'UK', 'yes', 18, 'Basic'],
  ['kiwitobes', 'France', 'yes', 19, 'Basic']],
 [['google', 'UK', 'no', 21, 'Premium'],
  ['(direct)', 'New Zealand', 'no', 12, 'None'],
  ['(direct)', 'UK', 'no', 21, 'Basic'],
  ['google', 'USA', 'no', 24, 'Premium'],
  ['digg', 'USA', 'no', 18, 'None'],
  ['google', 'UK', 'no', 18, 'None'],
  ['kiwitobes', 'UK', 'no', 19, 'None']])

In [5]:
def uniquecounts(rows):
    results = {}
    
    for row in rows:
        r = row[len(row) - 1]
        
        if r not in results:
            results[r] = 0
        
        results[r] += 1
    
    return results

In [9]:
def giniimpurity(rows):
    total = len(rows)
    
    counts = uniquecounts(rows)
    
    imp = 0
    
    for k1 in counts:
        p1 = float(counts[k1]) / total
        
        for k2 in counts:
            if k1 == k2:
                continue
            p2 = float(counts[k2])/total
            
            imp += p1*p2
    
    return imp

In [17]:
from math import log

def entropy(rows):
    log2 = lambda x: log(x)/log(2)
    
    results = uniquecounts(rows)
    
    ent = 0.0
    
    for r in results.keys():
        p = float(results[r] / len(rows))
        if p - 0.0<0.00000001:
            ent = ent - 0
        else:
            ent = ent - p*log(p+0.0000001,2)#log2(p)
    
    return ent

In [10]:
giniimpurity(my_data)

0.6400000000000001

In [18]:
entropy(my_data)

0.0

In [12]:
log(4,2)

2.0

In [19]:
def buildtree(rows , scoref = entropy):
    if len(rows) == 0:
        return decisionnode()
    
    current_score = scoref(rows)
    
    best_gain = 0.0
    best_criteria = None
    best_sets = None
    
    column_count = len(rows[0]) - 1
    
    for col in range(0 , column_count):
        column_values = {}
        
        for row in rows:
            column_values[row[col]] = 1
        
        for value in column_values.keys():
            (set1 , set2) = divideset(rows , col , value)
            
            p = float(len(set1))/len(rows)
            
            gain = current_score - p*scoref(set1)-(1-p)*scoref(set2)
            if gain>best_gain and len(set1)>0 and len(set2)>0:
                best_gain = gain
                best_criteria = (col , value)
                best_sets = (set1 , set2)
    
    if best_gain>0:
        trueBranch = buildtree(best_sets[0])
        falseBranch = buildtree(best_sets[1])
        
        return decisionnode(col=best_criteria[0],value=best_criteria[1],
                           tb=trueBranch , fb = falseBranch)
    else:
        return decisionnode(results=uniquecounts(rows))

In [21]:
tree = buildtree(my_data)

In [22]:
def printtree(tree , indent = ' '):
    if tree.results != None:
        print(str(tree.results))
    else:
        print(str(tree.col)+':'+str(tree.value)+'? ')
        
        print(indent+'T->')
        printtree(tree.tb , indent+'  ')
        print(indent+'F->')
        printtree(tree.fb , indent+'  ')

In [23]:
printtree(tree)

0:slashdot? 
 T->
{'None': 2}
 F->
{'None': 4, 'Premium': 3, 'Basic': 6}


In [24]:
def classify(observation , tree):
    if tree.results != None:
        return tree.results
    else:
        v = observation[tree.col]
        branch = None
        
        if isinstance(v , int) or isinstance(v , float):
            if v>= tree.value:
                branck = tree.tb
            else:
                branch = tree.fb
        else:
            if v == tree.value:
                branch = tree.tb
            else:
                branch = tree.fb
        
        return classify(observation , branch)

In [25]:
classify(['(direct)' , 'USA' , 'yes' , 5] , tree)

{'Basic': 6, 'None': 4, 'Premium': 3}

In [26]:
def prune(tree , mingain): #熵增益的阈值
    if tree.tb.results == None:
        prune(tree.tb , mingain)
    
    if tree.fb.results == None:
        prune(tree.fb , mingain)
    
    if tree.tb.results != None and tree.fb.results != None:
        tb , fb = [] , []
        
        for v , c in tree.tb.results.items():
            tb += [[v]] * c
        
        for v , c in tree.fb.results.items():
            fb += [[v]] * c
        
        delta = entropy(tb+fb) - (entropy(tb)+entropy(fb)/2)
        
        if delta<mingain:
            tree.tf , tree.fb = None , None
            tree.results = uniquecounts(tb+fb)

In [27]:
#对缺失值能够处理 的分类方法
def mdclassify(observation , tree):
    if tree.results != None:
        return tree.results
    else:
        v = observation[tree.col]
        if v == None:
            tr , fr = mdclassify(observation , tree.tb),mdclassify(observation , tree.fb)
            tcount = sum(tr.values())
            fcount = sum(fr.values())
            
            tw = float(tcount)/(tcount + fcount)
            fw = float(fcount)/(tcount + fcount)
            
            result = {}
            
            for k , v in tr.items():
                result[k] = v * tw
                
            for k , v in fr.items():
                if k not in result:
                    result[k] = 0
                    result[k] += v*fw
            return result
        else:
            if isinstance(v , int) or isinstance(v , float):
                if v>=tree.value:
                    branch = tree.tb
                else:
                    branch = tree.fb
            else:
                if v == tree.value:
                    branch = tree.tb
                else:
                    branch = tree.fb
            return mdclassify(observation , branch)

In [28]:
def variance(rows):
    if len(rows) == 0:
        return 0
    
    data = [float(row[len(row) - 1]) for row in rows]
    mean = sum(data)/len(data)
    variance = sum([(d-mean)**2 for d in data])/len(data)
    
    return variance